In [1]:
from datasets import load_from_disk, load_dataset, DatasetDict
from transformers import (
    TrainingArguments, Trainer, BatchEncoding,
    DistilBertTokenizerFast, DefaultDataCollator, DistilBertForQuestionAnswering
)
import torch
import numpy as np
import contextGenerator
import utils


checkpoint = 'distilbert-base-cased-distilled-squad'
tokenizer = DistilBertTokenizerFast.from_pretrained(checkpoint)
contextGen = contextGenerator.LuceneRetrieval()


try:
    ds = load_from_disk('../res/data/guess_train')
except:
    ds = load_dataset("community-datasets/qanta", "mode=first,char_skip=25")


/Users/alex/anaconda3/envs/QBAM/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Mar 05, 2025 2:52:40 AM org.apache.lucene.store.MemorySegmentIndexInputProvider <init>
INFO: Using MemorySegmentIndexInput with Java 21; to disable start with -Dorg.apache.lucene.store.MMapDirectory.enableMemorySegments=false


# Preprocessing the data 
Given how BERT is a extractive model it will attempt to highlight its prediction in the provided context. In other words our task is to fine tune the model to predict the start and end positions of the answer in the context.  
#### 1. Retreive context
For each question we will need a relevent document where the answer may exist. 

In [2]:
try: 
    ds['test']['context']
except:
    ds = ds.map(lambda x: {'context':  contextGen(x['full_question'], 1)[0]})

#### 2. Find the start and end postions
The contexts and questions are just strings to so we need to find the positions for the answers in the context. 

In [3]:
try: 
    ds['test']['char_pos']
except:
    ds = ds.map(lambda x: {'char_pos':  utils.term_char_index(x['answer'], x['context']['contents'])})

### 2. Tokenize context/question pair and find the token positions
Ensure the context comes first in the pair to align the character index with the token index. BERT limits the combined token count of context and question to 512. Since the context is capped at 400 words, this won’t cause issues, but we’ll use padding and truncation for consistency and edge cases.

In [4]:
unpack = lambda x, y, z: {"start_positions": x, "end_positions": y, "encodings": z}

def tokenize_row(row: dict, tokenizer) -> dict[str, BatchEncoding]:
    try: 
        encoding =  tokenizer(
            text = row['context']['contents'], 
            text_pair = row['full_question'], 
            padding = 'max_length', 
            truncation = 'only_first', 
            max_length = 512, 
            return_tensors = 'pt', 
            padding_side = 'right',
            return_length = True
            )
    except:
        cleaned = utils.clean_text(row['full_question'])
        encoding =  tokenizer(
            text = row['context']['contents'], 
            text_pair = cleaned, 
            padding = 'max_length', 
            truncation = 'only_first', 
            max_length = 512, 
            return_tensors = 'pt', 
            padding_side = 'right',
            return_length = True
            )
    start_pos = []
    end_pos = []
    for (x, y) in row['char_pos']:
        st = encoding.char_to_token(x)
        ed = encoding.char_to_token(y - 1)
        if st is None or ed is None:
            st = -1
            ed = -1
        start_pos.append(st)
        end_pos.append(ed)
    if len(start_pos) == 0:
        start_pos.append(-1)
        end_pos.append(-1)
    encoding.update({'start_positions': start_pos, 'end_positions': end_pos})
    return {"encodings": encoding}

try: 
    ds['test']['encodings']
except:
    ds = ds.map(lambda x: tokenize_row(x, tokenizer))
    
    

In [5]:
def equ_len_pad(encs): 
    # I want to find the largest list of start positions, from this pa all of rest to meet this size
    longest_len = max([len(x['start_positions']) for x in encs])
    for x in encs: 
        x_len = len(x['start_positions'])
        x['start_positions'] = x['start_positions'] + ([-1 for x in range(longest_len - x_len)])
        x['end_positions'] = x['end_positions'] + ([-1 for x in range(longest_len - x_len)])
        x['input_ids'] =  x['input_ids'][0]
        x['attention_mask'] = x['attention_mask'][0]
    return encs

train = equ_len_pad(ds['train']['encodings'])
val = equ_len_pad(ds['val']['encodings'])
test = equ_len_pad(ds['test']['encodings'])


In [6]:
# guessTrain = DatasetDict({
#     'train': ds['guesstrain'],
#     'val': ds['guessdev'],
#     'test': ds['guesstest'],
# })

# train = ds['train']
# val = ds['val']
# test = ds['test']
# ds.save_to_disk('../res/data/guess_train')

# Training

In [7]:
class BartTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        # generate model's guess
        outputs = model(input_ids = inputs['input_ids'] , attention_mask = inputs['attention_mask'])
        # find the model's predictions 
        start_yhat = torch.argmax(outputs['start_logits'], dim= 1)
        end_yhat = torch.argmax(outputs['end_logits'], dim= 1)
        # check if this is a possible target
        # if it is set it as the target, else choose a random valid target 
        start_target = []
        end_target = []

        valid_start_targets = []
        valid_end_targets = []

        for idx, x in enumerate(inputs['start_positions']): 
            cur_start_targ = []
            cur_end_targ = []

            for idx2, y in enumerate(x): 
                if y != -1:
                    cur_start_targ.append(y)
                    cur_end_targ.append(inputs['end_positions'][idx][idx2])

                else: 
                    break 
                
                
            valid_start_targets.append(cur_start_targ if cur_start_targ else [-1])
            valid_end_targets.append(cur_end_targ if cur_end_targ else [-1])          
            
        for x in range(len(inputs['input_ids'])):
            if start_yhat[x] in valid_start_targets[x]:
                start_target.append(start_yhat[x])                
                end_target.append(end_yhat[x])
            else: 
                ran_int = np.random.randint(len(valid_start_targets[x]))
                s_rand = valid_start_targets[x][ran_int]
                e_rand = valid_end_targets[x][ran_int]
     
                start_target.append(s_rand)
                end_target.append(e_rand)

        
        device = outputs['start_logits'].device
        start_target = torch.tensor(start_target, dtype=torch.long, device=device)
        end_target = torch.tensor(end_target, dtype=torch.long, device=device)

        # Compute loss
        loss_fct = torch.nn.CrossEntropyLoss(ignore_index=-1)
        start_loss = loss_fct(outputs['start_logits'], start_target)
        end_loss = loss_fct(outputs['end_logits'], end_target)
        
        total_loss = (start_loss + end_loss) / 2
        return (total_loss, outputs) if return_outputs else total_loss


In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained(checkpoint)
model = DistilBertForQuestionAnswering.from_pretrained(checkpoint)

if not torch.backends.mps.is_available():
    if not torch.backends.mps.is_built():
        print("MPS not available because the current PyTorch install was not "
              "built with MPS enabled.")
    else:
        print("MPS not available because the current MacOS version is not 12.3+ "
              "and/or you do not have an MPS-enabled device on this machine.")

else:
    mps_device = torch.device("mps")
    model.to(mps_device)


training_args = TrainingArguments(
    output_dir="res/models/guess_v1",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False
)

trainer = BartTrainer(
    model=model,
    args=training_args,                 
    train_dataset=train,
    eval_dataset=test,  
    data_collator=DefaultDataCollator(),  
    processing_class=tokenizer,    
)


trainer.train()

  2%|▏         | 500/30070 [09:01<8:44:21,  1.06s/it]

{'loss': 1.207, 'grad_norm': 2.1110000610351562, 'learning_rate': 4.9168606584635854e-05, 'epoch': 0.08}


  3%|▎         | 1000/30070 [17:57<8:36:13,  1.07s/it]

{'loss': 0.8761, 'grad_norm': 13.238821029663086, 'learning_rate': 4.83372131692717e-05, 'epoch': 0.17}


  5%|▍         | 1500/30070 [26:51<8:26:59,  1.06s/it]

...

 55%|█████▍    | 16500/30070 [8:57:04<19:23:48,  5.15s/it]

{'loss': 0.2703, 'grad_norm': 12.210532188415527, 'learning_rate': 2.256401729298304e-05, 'epoch': 2.74}


 57%|█████▋    | 17000/30070 [9:34:53<21:31:07,  5.93s/it]

{'loss': 0.3074, 'grad_norm': 14.998262405395508, 'learning_rate': 2.173262387761889e-05, 'epoch': 2.83}


 57%|█████▋    | 17154/30070 [9:46:11<8:43:58,  2.43s/it] 